In [1]:
import pandas as pd
import re
import os
import numpy as np

In [2]:
from models import rt

exchanges = set(i.id for i in rt.exchanges)

In [3]:
biolog_map = pd.read_csv('plate_to_bigg.csv', index_col=False)
biolog_map

,plate,ex_rxn,compound_name,exchange
0,PM1-A10,EX_tre(e),D-Trehalose,EX_tre_e
1,PM1-A11,EX_man(e),D-Mannose,EX_man_e
2,PM1-A12,EX_galt(e),Dulcitol,EX_galt_e
3,PM1-A2,EX_arab_L(e),L-Arabinose,EX_arab__L_e
4,PM1-A3,EX_acgam(e),N-Acetyl-DGlucosamine,EX_acgam_e
...,...,...,...,...
153,PM4-D6,EX_pser_L(e),O-Phospho-L-Serine,EX_pser__L_e
154,PM4-D7,EX_thrp(e),O-Phospho-L-Threonine,EX_thrp_e
155,PM4-D9,EX_3ump(e),Uridine-3’-monophosphate,EX_3ump_e
156,PM4-E1,EX_tyrp(e),O-Phospho-D-Tyrosine,EX_tyrp_e


In [4]:
fname = 'data_biolog_rhodo/Table_2_Multi-Omics Driven Metabolic Network Reconstruction and Analysis of Lignocellulosic Carbon Utilization in Rhodosporidium toruloides.XLSX'

bl_data = pd.read_excel(fname, sheet_name='Biolog')
bl_data['growth'] = bl_data['Average > 0.1 and all replicates greater than negative control']

In [5]:
bl_data['plate'] = bl_data.PlateType + '-' + bl_data.Well
bl_data.growth = bl_data.growth.astype(bool)
bl_data.head()

,PlateType,Experiment,Well,Compound,Average ABS (590nm - 750nm),All replicates > negtive control,Average > 0.1,Average > 0.1 and all replicates greater than negative control,pVal (T-test),growth,plate
0,PM1,Carbon,A1,Negative Control,0.086400,False,False,False,0.500000,False,PM1-A1
1,PM1,Carbon,A2,L-Arabinose,0.344767,True,True,True,0.000082,True,PM1-A2
2,PM1,Carbon,A3,N-Acetyl-DGlucosamine,0.068867,False,False,False,0.030610,False,PM1-A3
3,PM1,Carbon,A4,D-Saccharic Acid,0.071200,False,False,False,0.015131,False,PM1-A4
4,PM1,Carbon,A5,Succinic Acid,0.111200,True,True,True,0.081596,True,PM1-A5


In [6]:
carbon_plate = bl_data.loc[bl_data.PlateType.isin(['PM1', 'PM2'])].copy()
nitrogren_plate = bl_data.loc[bl_data.PlateType.isin(['PM3B'])].copy()
nitrogren_plate.plate = nitrogren_plate.plate.str.replace('B', '')
                           

In [7]:
nitrogren_plate[['plate', 'Compound', 'growth']]

,plate,Compound,growth
192,PM3-A1,Negative Control,False
193,PM3-A2,Ammonia,True
194,PM3-A3,Nitrite,True
195,PM3-A4,Nitrate,True
196,PM3-A5,Urea,True
...,...,...,...
283,PM3-H8,Gly-Asn,True
284,PM3-H9,Gly-Gln,True
285,PM3-H10,Gly-Glu,True
286,PM3-H11,Gly-Met,True


In [8]:
def merge_with_plate(plate):
    merged = pd.concat(
        [plate.set_index('plate'), biolog_map.set_index('plate')],
        axis=1,
        ignore_index=False
    )
    merged.dropna(subset=['growth', 'ex_rxn'], inplace=True)

    merged = merged.reset_index()
    merged#[['exchange', 'growth']]
    merged = subset_to_in_model(merged)[['exchange', 'growth']]
    merged['uptake'] = 10
    return merged

def subset_to_in_model(plate, model_ex=exchanges):
    return plate.loc[plate.exchange.isin(model_ex)]

carbon = merge_with_plate(carbon_plate)
nit = merge_with_plate(nitrogren_plate)
carbon.shape

(70, 3)

In [9]:
nit.shape

(37, 3)

In [10]:
carbon.to_csv('pm1_pm2_biolog.csv', index=False)
nit.to_csv('pm3b_biolog.csv', index=False)

## Gene ess. parsing

In [13]:
genes = set(i.id for i in rt.genes)
genes

{'10430',
 '15679',
 '10218',
 '16758',
 '10308',
 '9800',
 '14576',
 '10213',
 '12680',
 '14119',
 '15324',
 '14971',
 '10515',
 '16208',
 '9636',
 '15469',
 '11618',
 '15169',
 '11899',
 '16258',
 '11682',
 '9734',
 '14886',
 '14623',
 '8576',
 '13383',
 '9144',
 '14139',
 '10017',
 '15038',
 '9326',
 '14225',
 '16552',
 '16037',
 '9728',
 '15074',
 '11183',
 '8975',
 '12742',
 '16065',
 '9011',
 'COX2',
 '10540',
 '11017',
 '11973',
 '15287',
 '13190',
 '15911',
 '16092',
 '14195',
 '9425',
 '12570',
 '11375',
 '15059',
 '9746',
 '10592',
 '8441',
 '10953',
 '10292',
 '16505',
 '13288',
 '15180',
 '9241',
 '11403',
 '10551',
 '8859',
 '11971',
 '15328',
 '16359',
 '11837',
 '9923',
 '14960',
 '16775',
 '13706',
 '13935',
 '10812',
 '12795',
 '14465',
 '9708',
 '13708',
 '12099',
 '15746',
 '11741',
 '8456',
 '12122',
 '12096',
 'NAD6',
 '15756',
 '9250',
 '13419',
 '9571',
 '16423',
 '10007',
 '14022',
 '11389',
 '16404',
 '9274',
 '8853',
 '8460',
 '16323',
 '15807',
 '16666',
 '95

In [14]:
ge_data = pd.read_excel(fname, sheet_name='RB-TDNA Seq')
ge_data.RTO4_ID = ge_data.RTO4_ID.astype(str)
ge_data['genename'] = 'G_'+ge_data.RTO4_ID.astype(str)
ge_data[['genename', 'M9_Glucose']]

ge_data = ge_data.loc[ge_data.RTO4_ID.isin(genes)].copy()
ge_data

,RTO4_ID,M9_Glucose,YNB Glucose,YNB Glucose plus Arginine,YNB Glucose plus Methionine,YNB Glucose plus Dropout Complete,Fitness During Lipid Mobilization,YNB Oleic Acid,YNB Cellobiose,YNB_CSM_KPO4 Glucose,...,YNB_PO4_L-arabitol,YNB_PO4_xylitol,YNB_PO4_D-ribulose,YNB_PO4_D-xylulose,YNB_PO4_CSM_Galactose,YNB_PO4_CSM_Lactate,YNB_PO4_CSM_Valine,YNB_PO4_CSM_Leucine,YNB_PO4_CSM_Phenylalanine,genename
0,10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G_10000
7,10007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G_10007
10,10010,-0.130360,-0.558565,-0.332740,-0.260252,-0.383313,-0.030872,-0.611898,-0.097438,-0.035705,...,-0.732452,-0.836823,-0.490688,-0.534801,-0.035402,-0.427705,-0.143231,-0.240726,-0.376989,G_10010
12,10012,0.120928,-0.056685,-0.143451,-0.205516,-0.240177,-0.282986,-0.039371,0.021825,-0.185444,...,-0.185901,-0.347117,-0.043478,0.049640,-0.330369,-0.571357,-0.390520,-4.921371,-0.586259,G_10012
16,10017,-0.117902,0.001326,0.216430,0.169223,-0.072431,0.382748,0.455964,0.161420,0.012681,...,0.073421,0.076093,-0.266397,0.038548,0.024525,0.112220,-0.159979,-0.229817,-0.086100,G_10017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8306,9962,-0.102349,0.101140,0.044222,0.206130,0.142080,0.189212,-0.012657,-0.097314,0.004740,...,0.039896,0.240334,0.321752,0.089398,-0.254636,-0.723065,-0.622660,-0.047665,0.811095,G_9962
8323,9979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G_9979
8325,9981,-0.182969,-1.210120,-1.083946,-0.985173,-0.844407,-0.952909,-0.652234,0.200798,0.016013,...,-0.450650,-0.102575,-0.311876,-0.298921,-0.471137,-0.915556,-0.221405,-0.589374,-0.123179,G_9981
8334,9990,-0.272603,-0.184758,0.337242,0.525320,-0.020653,0.544806,-0.110552,-0.359997,0.187456,...,-4.105096,-4.463727,-0.667114,-4.072184,-0.077310,0.459774,0.053585,0.211604,-0.229242,G_9990


In [16]:
ge_data.dropna(subset=['M9_Glucose'], inplace=True)
ge_data

,RTO4_ID,M9_Glucose,YNB Glucose,YNB Glucose plus Arginine,YNB Glucose plus Methionine,YNB Glucose plus Dropout Complete,Fitness During Lipid Mobilization,YNB Oleic Acid,YNB Cellobiose,YNB_CSM_KPO4 Glucose,...,YNB_PO4_L-arabitol,YNB_PO4_xylitol,YNB_PO4_D-ribulose,YNB_PO4_D-xylulose,YNB_PO4_CSM_Galactose,YNB_PO4_CSM_Lactate,YNB_PO4_CSM_Valine,YNB_PO4_CSM_Leucine,YNB_PO4_CSM_Phenylalanine,genename
10,10010,-0.130360,-0.558565,-0.332740,-0.260252,-0.383313,-0.030872,-0.611898,-0.097438,-0.035705,...,-0.732452,-0.836823,-0.490688,-0.534801,-0.035402,-0.427705,-0.143231,-0.240726,-0.376989,G_10010
12,10012,0.120928,-0.056685,-0.143451,-0.205516,-0.240177,-0.282986,-0.039371,0.021825,-0.185444,...,-0.185901,-0.347117,-0.043478,0.049640,-0.330369,-0.571357,-0.390520,-4.921371,-0.586259,G_10012
16,10017,-0.117902,0.001326,0.216430,0.169223,-0.072431,0.382748,0.455964,0.161420,0.012681,...,0.073421,0.076093,-0.266397,0.038548,0.024525,0.112220,-0.159979,-0.229817,-0.086100,G_10017
28,10029,-0.020871,0.052095,0.224890,0.241546,-0.067308,0.622053,-0.412340,0.096512,-0.167850,...,-0.759319,-0.288770,-0.648312,0.551737,-0.069712,-0.414340,0.039014,-0.424238,-1.523899,G_10029
37,10038,-0.365987,-0.198609,-0.156681,0.504791,-0.298004,-0.330527,-0.754754,-0.425817,-0.064758,...,0.157148,0.081089,-0.082244,-0.037806,-0.397060,-0.226573,-0.212943,-0.605702,-0.036510,G_10038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8303,9959,-0.496589,-0.049252,-0.280461,-0.192801,-0.186293,0.094831,0.158215,-0.052001,-0.190215,...,-0.449143,-0.391230,-0.526536,-0.692617,-0.658693,-0.382288,-0.727372,-0.582871,0.019887,G_9959
8306,9962,-0.102349,0.101140,0.044222,0.206130,0.142080,0.189212,-0.012657,-0.097314,0.004740,...,0.039896,0.240334,0.321752,0.089398,-0.254636,-0.723065,-0.622660,-0.047665,0.811095,G_9962
8325,9981,-0.182969,-1.210120,-1.083946,-0.985173,-0.844407,-0.952909,-0.652234,0.200798,0.016013,...,-0.450650,-0.102575,-0.311876,-0.298921,-0.471137,-0.915556,-0.221405,-0.589374,-0.123179,G_9981
8334,9990,-0.272603,-0.184758,0.337242,0.525320,-0.020653,0.544806,-0.110552,-0.359997,0.187456,...,-4.105096,-4.463727,-0.667114,-4.072184,-0.077310,0.459774,0.053585,0.211604,-0.229242,G_9990


In [17]:
ge_data.loc[ge_data.M9_Glucose>.5, 'essential'] = True
ge_data.loc[ge_data.M9_Glucose<.5, 'essential'] = False


In [ ]:
ge_data['gene'] =  ge_data['RTO4_ID']
ge_data['comment'] = ''

In [28]:
ge_data.dtypes

RTO4_ID                               object
M9_Glucose                           float64
YNB Glucose                          float64
YNB Glucose plus Arginine            float64
YNB Glucose plus Methionine          float64
YNB Glucose plus Dropout Complete    float64
Fitness During Lipid Mobilization    float64
YNB Oleic Acid                       float64
YNB Cellobiose                       float64
YNB_CSM_KPO4 Glucose                 float64
YNB_CSM_KPO4 Xylose                  float64
YNB_CSM_KPO4 Arabinose               float64
YNB_CSM_KPO4 Acetate                 float64
YNB_CSM_KPO4 p-Coumarate             float64
YNB_CSM_KPO4 Ferulate                float64
YNB_PO4_Benzoate                     float64
YNB_PO4_Mannose                      float64
YNB_PO4_L-lyxose                     float64
YNB_PO4_D-arabitol                   float64
YNB_PO4_L-arabitol                   float64
YNB_PO4_xylitol                      float64
YNB_PO4_D-ribulose                   float64
YNB_PO4_D-

In [29]:
ge_data[['gene','essential', 'comment']]

,gene,essential,comment
10,10010,False,
12,10012,False,
16,10017,False,
28,10029,False,
37,10038,False,
...,...,...,...
8303,9959,False,
8306,9962,False,
8325,9981,False,
8334,9990,False,


In [30]:
import csv
ge_data[['gene','essential', 'comment']].to_csv('ge_m9_glucose.csv', index=False, quoting=csv.QUOTE_ALL)